In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.keras import utils
from sklearn.model_selection import StratifiedKFold
import category_encoders as ce
import numpy as np
import datetime
import itertools
from keras import regularizers

Using TensorFlow backend.


In [3]:
df = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

mapping_ord1 = {'Unknown': 0, 'Novice': 1, 'Contributor': 2, 'Expert': 3, 'Master': 4, 'Grandmaster': 5}
mapping_ord2 = {'Unknown': 0, 'Freezing': 1, 'Cold': 2, 'Warm': 3, 'Hot': 4, 'Boiling Hot': 5, 'Lava Hot': 6}
mapping_ord3 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_3'].fillna("0"))))])
mapping_ord4 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_4'].fillna("0"))))])
mapping_ord5 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_5'].fillna("0"))))])

def preprocess_data(df):
    df['ord_0'] = df['ord_0'].fillna(0)
    df['ord_1'] =  df['ord_1'].fillna('Unknown').map(mapping_ord1)
    df['ord_2'] =  df['ord_2'].fillna('Unknown').map(mapping_ord2)
    df['ord_3'] =  df['ord_3'].fillna('0').map(mapping_ord3)
    df['ord_4'] =  df['ord_4'].fillna('0').map(mapping_ord4)
    df['ord_5'] =  df['ord_5'].fillna('0').map(mapping_ord5)
    df['bin_3'] = df['bin_3'].fillna('U').map({"T": 1, "F": 0, "U": np.nan})
    df['bin_4'] = df['bin_4'].fillna('U').map({"Y": 1, "N": 0, "U": np.nan})
    return df
    
df = preprocess_data(df)
df_test = preprocess_data(df_test)

binary = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']
ordinal  = ['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']
low_card = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4']
high_card = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
date = ['day', 'month']
cyclic = ['day_sin', 'day_cos', 'month_sin', 'month_cos']

features = high_card + low_card + date + ordinal + binary

for c in tqdm(features):
    enc = preprocessing.LabelEncoder()
    enc.fit(pd.concat([df, df_test], sort=False)[c].fillna(-1).astype(str))
    df[c] = enc.transform(df[c].fillna(-1).astype(str))
    df_test[c] = enc.transform(df_test[c].fillna(-1).astype(str))

100%|██████████| 23/23 [00:20<00:00,  1.10it/s]


In [4]:
df.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,1,1,1,1,1,3,5,4,6,...,28,3,2,4,9,14,144,6,6,0
1,1,2,2,1,1,2,3,4,1,0,...,2113,3,5,3,11,17,59,7,10,0
2,2,1,2,1,1,1,3,0,4,1,...,0,3,0,1,6,8,9,5,12,0
3,3,0,1,1,1,1,3,1,4,4,...,2168,1,1,6,1,20,0,3,6,0
4,4,1,0,1,2,1,3,6,4,3,...,1748,3,5,2,14,20,138,5,4,0


In [5]:
def create_model(df, cols):    
    
    inputs = []
    outputs = []
        
    for c in cols:
        nunique = int(df[c].nunique())
        emb_dim = int(min(np.ceil((nunique)/2), 64))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(nunique + 1, emb_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.3)(out)
            
        out = layers.Reshape(target_shape=(emb_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(2, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=y)
    
    return model

In [6]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [7]:
def get_layer_output(X, model):
    layer = Model(inputs=model.input, outputs=model.get_layer("embeddings").output)
    out = emd_layer.predict(X)
    return out

In [11]:
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

folds = 50
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

es = EarlyStopping(monitor='val_auc', min_delta=0.0001, patience=5, verbose=1, mode='max',  
                   baseline=None,  restore_best_weights=True)

rlr = ReduceLROnPlateau(monitor='val_auc', factor=0.5, patience=3, min_lr=1e-6, mode='max', verbose=1)

df_test["target"] = 0

for train_idx, val_idx in tqdm(skf.split(df, df["target"])):
       
    X_train = [df.loc[train_idx, f].values for f in features]
    y_train = df.loc[train_idx, "target"].values
    
    X_val = [df.loc[val_idx, f].values for f in features]
    y_val = df.loc[val_idx, "target"].values
    
    model = create_model(df, features)
    model.compile(loss='binary_crossentropy', optimizer="adam", metrics=[auc])
    
    class_weights = class_weight.compute_class_weight('balanced', [0, 1], y_train)

    model.fit(X_train, utils.to_categorical(y_train), validation_data=(X_val, utils.to_categorical(y_val)),
              verbose=1, batch_size=1024, callbacks=[es, rlr], epochs=100, class_weight=class_weights)
       
    X_test = [df_test.loc[:, f].values for f in features]
    df_test["target"] += model.predict(X_test)[:, 1]
 
    K.clear_session()

0it [00:00, ?it/s]

Train on 588000 samples, validate on 12000 samples
Epoch 1/100
588000/588000 [==============================] - 15s 26us/sample - loss: 0.4701 - auc: 0.6981 - val_loss: 0.4050 - val_auc: 0.7811
Epoch 2/100
588000/588000 [==============================] - 12s 21us/sample - loss: 0.4081 - auc: 0.7702 - val_loss: 0.3970 - val_auc: 0.7864
Epoch 3/100
588000/588000 [==============================] - 13s 22us/sample - loss: 0.4029 - auc: 0.7789 - val_loss: 0.3949 - val_auc: 0.7878
Epoch 4/100
588000/588000 [==============================] - 13s 21us/sample - loss: 0.4003 - auc: 0.7827 - val_loss: 0.3979 - val_auc: 0.7850
Epoch 5/100
588000/588000 [==============================] - 12s 20us/sample - loss: 0.3984 - auc: 0.7857 - val_loss: 0.3969 - val_auc: 0.7858
Epoch 6/100
587776/588000 [============================>.] - ETA: 0s - loss: 0.3962 - auc: 0.7889
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
588000/588000 [==============================] - 13s 21u

1it [02:00, 120.22s/it]

Train on 588000 samples, validate on 12000 samples
Epoch 1/100
588000/588000 [==============================] - 14s 25us/sample - loss: 0.4731 - auc: 0.6974 - val_loss: 0.4084 - val_auc: 0.7872
Epoch 2/100
588000/588000 [==============================] - 11s 19us/sample - loss: 0.4082 - auc: 0.7702 - val_loss: 0.3972 - val_auc: 0.7942
Epoch 3/100
588000/588000 [==============================] - 11s 19us/sample - loss: 0.4030 - auc: 0.7784 - val_loss: 0.3942 - val_auc: 0.7933
Epoch 4/100
588000/588000 [==============================] - 12s 20us/sample - loss: 0.4001 - auc: 0.7828 - val_loss: 0.3976 - val_auc: 0.7923
Epoch 5/100
587776/588000 [============================>.] - ETA: 0s - loss: 0.3981 - auc: 0.7860
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
588000/588000 [==============================] - 11s 20us/sample - loss: 0.3981 - auc: 0.7859 - val_loss: 0.3982 - val_auc: 0.7905
Epoch 6/100
588000/588000 [==============================] - 11s 19u

2it [03:41, 114.43s/it]

...

48it [1:37:11, 131.54s/it]

Train on 588000 samples, validate on 12000 samples
Epoch 1/100
588000/588000 [==============================] - 16s 28us/sample - loss: 0.4716 - auc: 0.6966 - val_loss: 0.4099 - val_auc: 0.7785
Epoch 2/100
588000/588000 [==============================] - 13s 22us/sample - loss: 0.4081 - auc: 0.7707 - val_loss: 0.3984 - val_auc: 0.7869
Epoch 3/100
588000/588000 [==============================] - 13s 22us/sample - loss: 0.4030 - auc: 0.7788 - val_loss: 0.3984 - val_auc: 0.7871
Epoch 4/100
588000/588000 [==============================] - 13s 21us/sample - loss: 0.4001 - auc: 0.7831 - val_loss: 0.3986 - val_auc: 0.7865
Epoch 5/100
588000/588000 [==============================] - 13s 22us/sample - loss: 0.3979 - auc: 0.7864 - val_loss: 0.3997 - val_auc: 0.7855
Epoch 6/100
587776/588000 [============================>.] - ETA: 0s - loss: 0.3957 - auc: 0.7895
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
588000/588000 [==============================] - 13s 21u

49it [1:39:15, 129.45s/it]

Train on 588000 samples, validate on 12000 samples
Epoch 1/100
588000/588000 [==============================] - 16s 27us/sample - loss: 0.4732 - auc: 0.6966 - val_loss: 0.4112 - val_auc: 0.7785
Epoch 2/100
588000/588000 [==============================] - 13s 22us/sample - loss: 0.4080 - auc: 0.7706 - val_loss: 0.4003 - val_auc: 0.7819
Epoch 3/100
588000/588000 [==============================] - 13s 22us/sample - loss: 0.4029 - auc: 0.7789 - val_loss: 0.4008 - val_auc: 0.7825
Epoch 4/100
588000/588000 [==============================] - 13s 21us/sample - loss: 0.4004 - auc: 0.7825 - val_loss: 0.4008 - val_auc: 0.7815
Epoch 5/100
588000/588000 [==============================] - 13s 21us/sample - loss: 0.3979 - auc: 0.7859 - val_loss: 0.4012 - val_auc: 0.7815
Epoch 6/100
587776/588000 [============================>.] - ETA: 0s - loss: 0.3959 - auc: 0.7893
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
588000/588000 [==============================] - 13s 22u

50it [1:41:31, 121.84s/it]


In [12]:
df_test['target'] /= folds
df_test[['id', 'target']].to_csv('../data/submission_entity_encoding.csv', index=False)
!head -n 5 ../data/submission_entity_encoding.csv

id,target
600000,0.13013943
600001,0.28574654
600002,0.17581375
600003,0.15984708


# Combine submissions

In [10]:
import pandas as pd
df1 = pd.read_csv('../data/submission_lgbm.csv')
df2 = pd.read_csv('../data/submission_entity_encoding.csv')
df3 = pd.read_csv('../data/submission_wv.csv')

In [11]:
df1.head()

,id,target
0,600000,0.374122
1,600001,0.598371
2,600002,0.430493
3,600003,0.340466
4,600004,0.398753


In [12]:
df2.head()

,id,target
0,600000,0.130139
1,600001,0.285747
2,600002,0.175814
3,600003,0.159847
4,600004,0.158463


In [13]:
df3.head()

,id,target
0,600000,0.365186
1,600001,0.591646
2,600002,0.431123
3,600003,0.377906
4,600004,0.403135


In [14]:
df2["target"] = (0.3*df1["target"] + 0.7*df2["target"])

In [15]:
df2.to_csv('../data/submission_combine.csv', index=False)
!head -n 5 ../data/submission_combine.csv

id,target
600000,0.20333434656327662
600001,0.3795339504526317
600002,0.25221752576539436
600003,0.2140328608934602
